# Collecting and processing data from the Facebook Marketing API

*Authors*: Yuanmo He & Milena Tsvetkova

In this demo, we will collect location, age, and gender demographics for Facebook users who have a specific brand as interest.

# Data collection

## 1. Install `pysocialwatcher`

1. Open the Terminal in VS Code and install `git` in the virtual environment.

`conda install git`

2. Clone the project.

`git clone https://github.com/joaopalotti/pySocialWatcher.git`

3. Go to the project directory.

`cd [path to package directory that contains setup.py]`

4. Install the package.

`python setup.py install`

5. If you get errors that certain packages are missing, you should run in the Terminal.

  `conda install tabulate`
  
  `conda install coloredlogs`

In [8]:
# We use the library pysocialwatcher to query Facebook Marketing watcherAPI
# Details about the library: https://github.com/joaopalotti/pySocialWatcher
# Quick intro: https://goo.gl/WzE9ic
# Additional guidance: https://worldbank.github.io/connectivity_mapping/intro.html

from pysocialwatcher import watcherAPI
import pandas as pd
import json

## 2. Get your Facebook developer credentials

1. Create a **"Meta for Developers" account**.
     1. Go to https://developers.facebook.com/ and register with your Facebook account and phone number.
     2. Click on `New app` -> `Other` -> `Business account` -> `Marketing API` on the page *Add products to your app*
2. Get your **access token**.
     1. In the left panel select `Tools` -> `Get Access Token` -> Select the options starting with `ads_` -> Click on `Get Token`
     2. Copy the token in a csv file. The token is a long alphanumeric string.
3. Get your **application ID**.
     1. Go to https://www.facebook.com/business/ and click on `Create Advert`
     2. In the URL, copy the number after `act=` such as: https://www.facebook.com/ads/manager/creation/creation/?act=**952863473440**&pid=p1
     3. Copy that number in the csv file as a second column.

In [9]:
watcher = watcherAPI(api_version="15.0")

# Access your Facebook credentials from the file fb_credentials.csv 
# Access Token goes in the first column, Application ID is in the second column 
watcher.load_credentials_file("data/fb_credentials.csv")

## 3. Get the brands' interest ids

In [10]:
# Collect the Facebook interest ids for the brands
brands_on_twitter = pd.read_csv("data/brands_on_twitter.csv")
brands = brands_on_twitter['brand'][:5]  # Just the first 5 for the demo
rowlst = []
errorlst = []

# We'll catch any errors due to names not being found
for i in range(len(brands)):   
    try:
        rowlst.append(dict(watcher.get_interests_given_query(brands[i]).loc[0]))
    except Exception as e:
        errorlst.append({i: e})
    print(i)

0
1
2
3
4


In [11]:
print('Found:', len(rowlst)) 
print('Errors:', len(errorlst)) 

# No errors here but otherwise, you need to resolve them manually
# Errors could be due to different spelling or no/deleted account

Found: 5
Errors: 0


In [13]:
# Save interest id and check errors manually 
fb_interest_id = pd.DataFrame(rowlst)
error_ids = [list(i.keys())[0] for i in errorlst]
fb_interest_id.insert(0, 'brand', brands.drop(error_ids).reset_index(drop = True))
fb_interest_id = fb_interest_id.rename(columns = {'name': "FB_name"})
fb_interest_id.to_excel('data/FB_interest_id_raw.xlsx', index = False) 

# Label the weird brands in FB_interest_id_raw.xlsx,
# manually check and modify in FB_interest_id.xlsx.
# Also manually add the ones that cannot be found automatically.

## 4. Create `json` files for data collection
We use code to create the list of interest ids. Age, gender, and location can be easily written manually. 

In [14]:
fb_interest_id = pd.read_excel("data/FB_interest_id_raw.xlsx")

with open ("data/interest_list.json", "a") as file:
    for index, row in fb_interest_id.iterrows():
        json.dump({"or": [row['id']], "name": row['brand']}, file)
        file.write(',\n')

We are not sharing the interest ids of the brands (the content of interest_list.json), as it is not clear from Facebook whether we have the rights to share them. But you should be able to get them with the code provided so far. Then, use the json files as intructed by pysocialwatcher https://github.com/joaopalotti/pySocialWatcher to collect the data. The following cell provides the example json contents to be saved in the correponding json files.

## 5. Collect data 


In [ ]:
to_collect_list = ["data/collecting_urban.json"] #, "data/collecting_gender.json", "data/collecting_age.json"]
collected_files = ["data/collected_urban.csv"] #, "data/collected_gender.csv", "data/collected_age.csv"]

for i in range(len(to_collect_list)):
    
    collecting = to_collect_list[i]
    collected = collected_files[i]
    
    print("start", collecting, "---")
    watcher = watcherAPI(api_version="15.0", outputname = collected)
    watcher.load_credentials_file("data/FB_credentials.csv")
    
    df = watcher.run_data_collection(collecting, remove_tmp_files = True)
    
    print(collected, "end---")

# Porcessing data

Process the collected data and in the end get two files representing the upper and lower bounds for monthly active users. 

In [ ]:
def clean_pivot(csvfile, key_column):
    """
    Input the file name of the csv file and the column name of the variable (age, gender, etc.)
    return the cleaned and pivoted version of dataframe for upper and lower mau
    """
    
    df = pd.read_csv(csvfile, index_col = 0)
    df = df[['interests', key_column, 'mau_audience_upper_bound', 'mau_audience_lower_bound']]
    df = df.assign(id = df['interests'].str.extract("\[([0-9]+)\]")) 
    
    upper = df[['id', 'mau_audience_upper_bound', key_column]]
    upper = upper.pivot(index = 'id', columns = key_column, values = 'mau_audience_upper_bound')
    upper.columns.name = None
    
    lower = df[['id', 'mau_audience_lower_bound', key_column]]
    lower = lower.pivot(index = 'id', columns = key_column, values = 'mau_audience_lower_bound')
    lower.columns.name = None
    
    return upper, lower

In [ ]:
# Process different categories individually
upper_geo, lower_geo = clean_pivot("collected_urban.csv", 'geo_locations')
upper_geo.columns = ['all_US', 'urban']
lower_geo.columns = ['all_US', 'urban']

upper_gender, lower_gender = clean_pivot("collected_gender.csv", 'genders')
upper_gender.columns = ['male', 'female']
lower_gender.columns = ['male', 'female']

upper_age, lower_age = clean_pivot("collected_age.csv", 'ages_ranges')
upper_age.columns = ['age18_24', 'age25_34', 'age35_44', 'age45_54', 'age55_64', 'age65plus']
lower_age.columns = ['age18_24', 'age25_34', 'age35_44', 'age45_54', 'age55_64', 'age65plus']

In [ ]:
# Put together in two different dataframes
upper_mau = pd.concat([upper_geo, upper_age, upper_gender], axis = 1)
lower_mau = pd.concat([lower_geo, lower_age, lower_gender], axis = 1)

FB_interet_id = pd.read_excel("data/FB_interest_id_raw.xlsx", dtype = {'id': str})[['brand', 'id']]
upper_mau = pd.merge(FB_interet_id, upper_mau, on = 'id').drop(columns = ['id'])
lower_mau = pd.merge(FB_interet_id, lower_mau, on = 'id').drop(columns = ['id'])

In [ ]:
# Find invalid cases where the upper and lower bound for all US are both 1000
not_valid = pd.merge(upper_mau[['brand', 'all_US']], lower_mau[['brand', 'all_US']], on = 'brand')
not_valid = not_valid.set_index('brand')
not_valid = not_valid == 1000
not_valid = list(not_valid[not_valid.sum(axis = 1) == 2].index.values)

# Delete invalid cases
upper_mau = upper_mau[~upper_mau['brand'].isin(not_valid)]
lower_mau = lower_mau[~lower_mau['brand'].isin(not_valid)]

# Save data for analysis
upper_mau.to_csv('data/upper_mau.csv', index = False)
lower_mau.to_csv('data/lower_mau.csv', index = False)